In [785]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [786]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
r = requests.get(url)
r_text = r.text

In [787]:
soup = BeautifulSoup(r_text, 'html.parser')

In [788]:
soup = soup.body.table.tbody

In [789]:
df = pd.DataFrame(columns=range(0,3), index = range(0,len(soup.find_all('tr'))))

row_marker = 0
for row in soup.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        df.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1

df.head()

,0,1,2
0,NaN,NaN,NaN
1,M1A,Not assigned,Not assigned\n
2,M2A,Not assigned,Not assigned\n
3,M3A,North York,Parkwoods\n
4,M4A,North York,Victoria Village\n


In [790]:
df = df.replace(r'\n',' ', regex=True)
df = df.replace(r'Not assigned',np.NaN,regex=True)

df.rename(columns={0:'PostalCode',1:'Borough',2:'Neighborhood'},inplace=True)
df.dropna(axis=0,subset=['Borough'],inplace=True)
df.reset_index(drop=True,inplace=True)

df['Neighborhood'] = df['Neighborhood'].replace(np.NaN,df['Borough'])

df.head(10)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [791]:
df_gr = df.groupby(by=['PostalCode','Borough']).Neighborhood.unique().apply(lambda x: ', '.join(x)).reset_index()
df_gr.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park"
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge"
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village W..."
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


In [792]:
df_gr.shape

(103, 3)